In [58]:
import datetime, mmap,os,re, sys, decimal
import csv, getpass, os, random, sys, glob
import numpy as np 
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import scipy as sp

In [97]:
# load in the retrieval and encoding files
ret_file = pd.read_csv('Data/555_2017_Mar_02_1437_StressMem_ret.csv')
enc_file = pd.read_csv('Data/555_2017_Mar_02_1140_StressMem_enc.csv')

In [98]:
ret_file.head()

,subject,phase,session,trial,item,enc_task,enc_response,enc_RT,valence,letters,...,ret_options_duration,task_resp,task_rt,task_onset,task_duration,task_prompt_onset,task_prompt_duration,task_options_onset,task_options_duration,iti_durs
0,555,experiment,1,1,skilled,NaN,NaN,NaN,1,7,...,2.536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1002.0
1,555,experiment,1,2,inferior,NaN,NaN,NaN,0,8,...,3.070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2004.0
2,555,experiment,1,3,radiant,NaN,NaN,NaN,1,7,...,3.138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1002.0
3,555,experiment,1,4,captive,Describes,no,0.618,0,7,...,2.602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,501.0
4,555,experiment,1,5,happy,NaN,NaN,NaN,1,5,...,2.669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2004.0


In [99]:
ret_file.tail()

,subject,phase,session,trial,item,enc_task,enc_response,enc_RT,valence,letters,...,ret_options_duration,task_resp,task_rt,task_onset,task_duration,task_prompt_onset,task_prompt_duration,task_options_onset,task_options_duration,iti_durs
195,555,experiment,2,196,reject,Positive,yes,0.768,0,6,...,9.508,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1002.0
196,555,experiment,2,197,dreamer,Positive,yes,0.486,1,7,...,3.254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2004.0
197,555,experiment,2,198,insulting,Positive,yes,0.551,0,9,...,3.024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1002.0
198,555,experiment,2,199,uninvited,NaN,NaN,NaN,0,9,...,10.045,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2004.0
199,555,experiment,2,200,safe,Positive,no,6.258,1,4,...,10.044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1002.0


In [100]:
# Are any words repeated?

print(ret_file.item.count())
print(ret_file.item.nunique()) 

200
200


In [101]:
# Are the two blocks balanced on old/new?

ret_file.groupby(['session', 'status'])['subject'].count()

session  status
1        new       50
         old       50
2        new       50
         old       50
Name: subject, dtype: int64

In [102]:
# Are the two blocks balanced on valence?

ret_file.groupby(['session', 'valence'])['subject'].count()

session  valence
1        0          50
         1          50
2        0          50
         1          50
Name: subject, dtype: int64

In [103]:
# Are the two blocks balanced on task?

ret_file.groupby(['session','enc_task'])['subject'].count()

session  enc_task 
1        Describes    25
         Positive     25
2        Describes    25
         Positive     25
Name: subject, dtype: int64

In [104]:
# Is everything balanced within these?

ret_file.groupby(['session','status','valence','enc_task'])['subject'].count()

session  status  valence  enc_task 
1        old     0        Describes    13
                          Positive     12
                 1        Describes    12
                          Positive     13
2        old     0        Describes    12
                          Positive     13
                 1        Describes    13
                          Positive     12
Name: subject, dtype: int64

In [128]:
# Are the words being labeled as old/new correctly?

old_words = []
old_words = ret_file[ret_file.status == 'old']['item']
old_words

for word in old_words:
    if word in enc_file['word'].unique():
        pass
    else:
        print ('***WORD IS MISLABELED****')

In [138]:
list(ret_file)

['subject',
 'phase',
 'session',
 'trial',
 'item',
 'enc_task',
 'enc_response',
 'enc_RT',
 'valence',
 'letters',
 'frequency',
 'concreteness',
 'part_of_speech',
 'imageability',
 'status',
 'recog_resp',
 'recog_rt',
 'recog_result',
 '    old_new_onset',
 'old_new_duration',
 'ret_stim_onset',
 'ret_stim_duration',
 'ret_options_onset',
 'ret_options_duration',
 'task_resp',
 'task_rt',
 'task_onset',
 'task_duration',
 'task_prompt_onset',
 'task_prompt_duration',
 '    task_options_onset',
 'task_options_duration',
 'iti_durs']

In [144]:
# How do the no response trials look?

no_response = ret_file[ret_file.recog_resp.isnull()]
no_response = no_response[['item','recog_resp','recog_rt','recog_result','    old_new_onset','old_new_duration','ret_stim_onset','ret_stim_duration','ret_options_onset','ret_options_duration','task_resp']]
no_response

,item,recog_resp,recog_rt,recog_result,old_new_onset,old_new_duration,ret_stim_onset,ret_stim_duration,ret_options_onset,ret_options_duration,task_resp
50,victim,NaN,NaN,NaN,521.218,0.994,522.211,3.004,525.216,10.027,NaN
51,witty,NaN,NaN,NaN,539.250,0.998,540.248,3.003,543.251,10.029,NaN
53,value,NaN,NaN,NaN,572.619,0.999,573.618,3.002,576.621,10.011,NaN
54,affectionate,NaN,NaN,NaN,591.674,0.998,592.672,3.003,595.675,10.028,NaN
65,confident,NaN,NaN,NaN,715.493,1.014,716.507,3.004,719.510,10.010,NaN
67,comfortable,NaN,NaN,NaN,740.619,1.031,741.651,3.003,744.654,10.030,NaN
97,excited,NaN,NaN,NaN,1026.749,1.064,1027.813,3.170,1030.983,10.011,NaN
99,selfish,NaN,NaN,NaN,1054.946,1.064,1056.010,3.170,1059.180,10.011,NaN
103,hardship,NaN,NaN,NaN,51.104,0.997,52.101,3.004,55.105,10.010,NaN
104,romantic,NaN,NaN,NaN,70.125,0.997,71.122,3.036,74.158,10.011,NaN


In [106]:
# Do the recog RTs look reasonable? (should all be less than 10s)

ret_file.recog_rt.describe()

count    167.000000
mean       1.626802
std        1.865097
min        0.060000
25%        0.534500
50%        0.807000
75%        1.802500
max        8.451000
Name: recog_rt, dtype: float64

In [107]:
# and for task?

ret_file.task_rt.describe()

count    17.000000
mean      1.212588
std       1.330413
min       0.160000
25%       0.536000
50%       0.665000
75%       1.096000
max       5.502000
Name: task_rt, dtype: float64